# 🧠 Run SQL Challenges with DuckDB

This notebook loads the CSV datasets and runs each SQL challenge using DuckDB — all in one place.


In [1]:
# pip install duckdb

import duckdb
con = duckdb.connect()

In [2]:
# Load all CSVs as tables
# Load all CSVs as tables using read_csv_auto()
con.execute("CREATE TABLE customers AS SELECT * FROM read_csv_auto('../datasets/customers.csv');")
con.execute("CREATE TABLE orders AS SELECT * FROM read_csv_auto('../datasets/orders.csv');")
con.execute("CREATE TABLE products AS SELECT * FROM read_csv_auto('../datasets/products.csv');")

## 🧩 Challenge 01: Top Customers by Revenue

This query identifies the top 5 customers based on total revenue generated

In [3]:
con.sql("""
    SELECT
        customer_id,
        COUNT(order_id) AS total_orders,
        SUM(order_amount) AS total_revenue
    FROM orders
    GROUP BY customer_id
    ORDER BY total_revenue DESC
    LIMIT 5
""").df()

,customer_id,total_orders,total_revenue
0,1,3,3200.0
1,3,3,2150.0
2,5,1,800.0
3,2,2,500.0
4,4,1,200.0


## 📈 Challenge 02: Monthly Revenue and MoM Growth

This query calculates total revenue per month and computes the month-over-month (MoM) percentage change.

In [4]:
con.sql("""
WITH monthly_revenue AS (
    SELECT 
        DATE_TRUNC('month', order_date) AS month,
        SUM(order_amount) AS total_revenue
    FROM orders
    GROUP BY month
),
revenue_with_growth AS (
    SELECT 
        month,
        total_revenue,
        ROUND(
            (total_revenue - LAG(total_revenue) OVER (ORDER BY month)) 
            / NULLIF(LAG(total_revenue) OVER (ORDER BY month), 0) * 100,
        2) AS mom_growth
    FROM monthly_revenue
)
SELECT * FROM revenue_with_growth
""").df()

,month,total_revenue,mom_growth
0,2024-01-01,1200.0,NaN
1,2024-02-01,1100.0,-8.33
2,2024-03-01,1350.0,22.73
3,2024-04-01,1000.0,-25.93
4,2024-05-01,200.0,-80.00
5,2024-06-01,1200.0,500.00
6,2024-07-01,800.0,-33.33


## 🥇 Challenge 03: Best-Selling Products

This query identifies the top 5 best-selling products by total revenue.  
It joins the `orders` and `products` tables and aggregates the data to determine which products contributed the most revenue overall.

In [5]:
con.sql("""
    SELECT
        p.product_id,
        p.product_name,
        COUNT(o.order_id) AS total_orders,
        SUM(o.order_amount) AS total_revenue
    FROM orders o
    JOIN products p ON o.product_id = p.product_id
    GROUP BY p.product_id, p.product_name
    ORDER BY total_revenue DESC
    LIMIT 5
""").df()

,product_id,product_name,total_orders,total_revenue
0,1,Laptop,3,3600.0
1,2,Smartphone,3,2400.0
2,5,Headphones,2,400.0
3,3,Desk,1,300.0
4,4,Chair,1,150.0


## 🌍 Challenge 04: Region Sales Share

This query calculates the percentage of total revenue contributed by each customer region.  
It helps visualize the geographic distribution of sales, useful for market analysis and resource allocation.

In [6]:
con.sql("""
    SELECT
        c.region,
        SUM(o.order_amount) AS total_revenue,
        ROUND(SUM(o.order_amount) * 100.0 / SUM(SUM(o.order_amount)) OVER (), 2) AS revenue_share_pct
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.region
    ORDER BY revenue_share_pct DESC
""").df()


,region,total_revenue,revenue_share_pct
0,North,4000.0,58.39
1,West,2150.0,31.39
2,South,500.0,7.30
3,East,200.0,2.92


## 🔁 Challenge 05: Repeat vs New Customers

This query classifies each order as being placed by a **new** or **repeat** customer  
and calculates the total revenue contributed by each group.  

It’s useful for measuring customer loyalty and acquisition effectiveness.

In [7]:
con.sql("""
    WITH customer_orders AS (
        SELECT
            customer_id,
            order_id,
            order_date,
            MIN(order_date) OVER (PARTITION BY customer_id) AS first_order_date
        FROM orders
    ),
    labeled_orders AS (
        SELECT
            customer_id,
            order_id,
            order_date,
            CASE 
                WHEN order_date = first_order_date THEN 'New'
                ELSE 'Repeat'
            END AS customer_type
        FROM customer_orders
    )
    SELECT
        customer_type,
        COUNT(o.order_id) AS total_orders,
        SUM(order_amount) AS total_revenue
    FROM labeled_orders lo
    JOIN orders o ON lo.order_id = o.order_id
    GROUP BY customer_type
    ORDER BY total_revenue DESC
""").df()

,customer_type,total_orders,total_revenue
0,New,5,3700.0
1,Repeat,5,3150.0


## 📦 Challenge 06: Product Popularity Index

This query evaluates how popular each product is by counting the number of customers who ordered it.  
It helps identify which products have the widest appeal and customer reach.

In [8]:
con.sql("""
    SELECT
        p.product_id,
        p.product_name,
        COUNT(DISTINCT o.customer_id) AS unique_customers,
        COUNT(o.order_id) AS total_orders
    FROM orders o
    JOIN products p ON o.product_id = p.product_id
    GROUP BY p.product_id, p.product_name
    ORDER BY unique_customers DESC, total_orders DESC
""").df()

,product_id,product_name,unique_customers,total_orders
0,2,Smartphone,3,3
1,1,Laptop,2,3
2,5,Headphones,2,2
3,3,Desk,1,1
4,4,Chair,1,1


## 💰 Challenge 07: Customer Lifetime Value (LTV)

This query estimates **Customer Lifetime Value (LTV)** by summing the total order value per customer.  
It is a key metric for understanding which customers generate the most revenue over time.

In [9]:
con.sql("""
    SELECT
        c.customer_id,
        c.customer_name,
        SUM(o.order_amount) AS total_revenue,
        COUNT(o.order_id) AS total_orders,
        ROUND(AVG(o.order_amount), 2) AS avg_order_value
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.customer_name
    ORDER BY total_revenue DESC
""").df()

,customer_id,customer_name,total_revenue,total_orders,avg_order_value
0,1,Ana Smith,3200.0,3,1066.67
1,3,Luisa Diaz,2150.0,3,716.67
2,5,Sofia López,800.0,1,800.00
3,2,John Doe,500.0,2,250.00
4,4,David Chen,200.0,1,200.00


## 🌍 Challenge 08: Order Frequency by Region

This query calculates the average number of orders per customer, broken down by region.  
It provides insights into which regions have the most engaged customers.

In [10]:
con.sql("""
    SELECT
        c.region,
        COUNT(o.order_id) AS total_orders,
        COUNT(DISTINCT c.customer_id) AS total_customers,
        ROUND(COUNT(o.order_id) * 1.0 / COUNT(DISTINCT c.customer_id), 2) AS avg_orders_per_customer
    FROM orders o
    JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.region
    ORDER BY avg_orders_per_customer DESC
""").df()

,region,total_orders,total_customers,avg_orders_per_customer
0,West,3,1,3.0
1,North,4,2,2.0
2,South,2,1,2.0
3,East,1,1,1.0


## 📅 Challenge 09: Top Months by Order Volume

This query identifies the months with the highest number of orders.  
It can be used to detect seasonal trends or spikes in demand.

In [11]:
con.sql("""
    SELECT
        DATE_TRUNC('month', order_date) AS month,
        COUNT(order_id) AS total_orders
    FROM orders
    GROUP BY month
    ORDER BY total_orders DESC
    LIMIT 5
""").df()

,month,total_orders
0,2024-02-01,2
1,2024-04-01,2
2,2024-03-01,2
3,2024-05-01,1
4,2024-01-01,1


## 🔍 Challenge 10: RFM Analysis

This query performs an **RFM (Recency, Frequency, Monetary)** analysis on the customer base.  
It helps to segment customers based on their behavior and purchasing activity.

In [12]:
con.sql("""
    WITH rfm AS (
        SELECT
            customer_id,
            MAX(order_date) AS last_order,
            COUNT(order_id) AS frequency,
            SUM(order_amount) AS monetary
        FROM orders
        GROUP BY customer_id
    )
    SELECT
        customer_id,
        CURRENT_DATE - last_order AS recency,
        frequency,
        monetary
    FROM rfm
    ORDER BY recency ASC, frequency DESC, monetary DESC
""").df()

,customer_id,recency,frequency,monetary
0,3,316,3,2150.0
1,1,336,3,3200.0
2,2,373,2,500.0
3,5,390,1,800.0
4,4,405,1,200.0
